In [7]:
import pandas as pd 
import numpy as np
import re
import sys
import os
import nltk
from nltk import sent_tokenize 
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB 
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api 
from gensim.utils import simple_preprocess
from tqdm import tqdm 
from bs4 import BeautifulSoup
nltk.download('stopwords')

cv = CountVectorizer()
tfid = TfidfVectorizer()
lemmatizer = WordNetLemmatizer()

kindle = pd.read_csv('all_kindle_review.csv')


kindle.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [8]:
kindle = kindle[['reviewText', 'rating']]
kindle.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [9]:
corpus = []

for i in range(len(kindle['reviewText'])):
    reviews = re.sub('[^a-z A-Z 0-9-]+', ' ', kindle['reviewText'][i])
    reviews = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ', reviews)
    reviews = reviews.lower()
    reviews = BeautifulSoup(reviews, 'lxml').get_text()
    reviews = reviews.split()
    reviews = [lemmatizer.lemmatize(word) for word in reviews]
    reviews = ' '.join(reviews)
    corpus.append(reviews) 

In [10]:
corpus

['jace rankin may be short but he s nothing to mess with a the man who wa just hauled out of the saloon by the undertaker know now he s a famous bounty hunter in oregon in the 1890s who when he shot the man in the saloon just finished a year long quest to avenge his sister s murder and is now trying to figure out what to do next when the snotty-nosed farm boy he just rescued from a gang of bully offer him money to kill a man who forced him off his ranch he reluctantly agrees to bring the man to justice but not to kill him outright but first he need to tell his sister s widower the news kyla kyle springer bailey ha been riding the trail and sleeping on the ground for the past month while trying to find jace she want revenge on the man who killed her husband and took her ranch amongst other crime and she s not so keen on the detour jace want to take but she realizes she s out of option so she hide behind her boy persona a best she can and try to keep pace when a confrontation along the w

In [11]:
kindle['reviewText'] = kindle['rating'].apply(lambda x: 0 if x < 3 else 1)
y = kindle['reviewText']

In [12]:
x_train, x_test, y_train, y_test = train_test_split(corpus, y, test_size = 0.2)
x_train = cv.fit_transform(x_train).toarray()
x_test = cv.transform(x_test).toarray()  

bowmod = MultinomialNB().fit(x_train, y_train)
y_pred = bowmod.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8275
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       808
           1       0.88      0.85      0.87      1592

    accuracy                           0.83      2400
   macro avg       0.81      0.81      0.81      2400
weighted avg       0.83      0.83      0.83      2400



In [14]:
x_train, x_test, y_train, y_test = train_test_split(corpus, y, test_size = 0.2)
x_train = tfid.fit_transform(x_train).toarray()
x_test = tfid.transform(x_test).toarray()

tfidmod = GaussianNB().fit(x_train, y_train)
y_pred = tfidmod.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.5666666666666667
              precision    recall  f1-score   support

           0       0.41      0.62      0.49       826
           1       0.73      0.54      0.62      1574

    accuracy                           0.57      2400
   macro avg       0.57      0.58      0.56      2400
weighted avg       0.62      0.57      0.58      2400

